In [112]:
%load_ext autoreload
%autoreload 2

## Import necessary modules
import os,sys
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator, AutoLocator, MultipleLocator, FormatStrFormatter, ScalarFormatter, FuncFormatter
import numpy as np
import datetime, calendar
from datetime import timedelta
import matplotlib.patches as mpatches
from itertools import tee
from traitlets import traitlets
import matplotlib.gridspec as gridspec

sys.path.append(os.path.abspath('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/entwuerfe/pyplots/lib/'))
from ce_funclib import determine_kernzeit as dtkz
from ce_funclib import continuity_check, decminutes_to_mmss as dec2min

from ipywidgets import widgets, interact, interactive, fixed, interact_manual, Layout
from IPython.display import display

%matplotlib tk

## Import data frome pickle generated from muß ein file mit agentenstats sein, wird generiert von Datensammler_Agenten.ipynb
pklpth='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/datenhalde/datapickles/Rohdaten_Agenten_aktuell.pkl'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Funktionen

In [113]:
def add_averages(frame):

    frame['avtt']=frame['tt_float']/frame['be']
    frame['avht']=frame['ht_float']/frame['be']
    frame['avacw']=frame['acw_float']/frame['be']
    
    colorder_with_abs=['an','be','vl','avht','avtt','avacw','ht_float','tt_float','acw_float']
    frame_with_abs=frame[colorder_with_abs]
    
    #colorder_av_only=['an','be','vl','avht','avtt','avacw']
    #frame_only_av=frame[colorder_av_only].fillna(0).unstack('bz')  # die Bearbeitungszeit soll hoch in die Spalten
    
    return frame_with_abs.fillna(0)

## Daten und Dataframe

In [114]:
big_frame=pd.read_pickle(pklpth)  # wo kommt der pickle her und wie generiere ich einen aktualisierten? -> Datensammler.py
big_frame[['ht_float','tt_float']]=big_frame[['ht_float','tt_float']].apply(pd.to_numeric)
big_frame['acw_float']=big_frame['ht_float']-big_frame['tt_float']
big_frame['calweeks']=big_frame.index.strftime('%Y-%V')

In [125]:
## Zeitraum für den plot wählen und filtern
von_bis=(datetime.date(2018,1,1),datetime.date(2018,4,19))
zeitraum=big_frame.loc[(big_frame.index.date >= von_bis[0]) & (big_frame.index.date <= von_bis[1])].copy()

In [126]:
zeitraum=zeitraum.groupby([pd.Grouper('agent'),pd.Grouper('bz')]).sum().unstack('bz',fill_value=0).stack('bz')
g = zeitraum.sum(level=['agent']) # einen Zwischenframe erstellen, der nur die Kern- und Nebenzeit summiert
zeitraum=zeitraum.append(g.assign(bz='g').set_index('bz', append=True)).sort_index() # die Summe dem alten Frame beigeben

In [127]:
zeitraum=add_averages(zeitraum)

In [128]:
zindex=zeitraum.xs('g',level='bz').sort_values('avht').index   # nur gesamtzeit sortiert nach angenommenen, davon der index
zeitraum=zeitraum.reindex(zindex,level=0) # frame danach sortiert

In [133]:
def agentplot(frame,vonbis):
    
    gcol,lostcol,verbcol,ncol,kcol,ncol2="#777777","#AC003A","#008EC4","#000000","#4F7DFF",'#424242'
    
    fig = plt.figure(figsize=(10,7))
    ax1 = plt.subplot2grid((1, 3), (0, 1), colspan=2)
    ax2 = plt.subplot2grid((1, 3), (0, 0), colspan=1)
    
    d_ix=frame.index.get_level_values(0).unique()
    agents_ticks=range(len(d_ix))
    
    ####################################################
    ### subplot für die Zeiten
    
    ## y-ticks nach Agenten
    ax1.set_yticks(agents_ticks)
    ax1.set_yticklabels(d_ix)
            
    ## locators und formatters für die rechte x-achse
    ax1.xaxis.set_major_locator(MultipleLocator(1))
    ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax1.xaxis.set_major_formatter(FuncFormatter(dec2min))
    ax1.xaxis.set_minor_formatter(FuncFormatter(dec2min))    # Die Dezimalzahlen der Y-Achse sollen labels im MM:SS Format erhalten
    
    for label in ax1.get_xticklabels(which='both'):          # lösung mit loop scheint am einfachsten zu sein...
        label.set_rotation(75) 
    for label in ax1.get_xticklabels(which='major'):
        label.set_fontsize(7)
    for label in ax1.get_xticklabels(which='minor'):
        label.set_fontsize(6)
    
    ## Hilfslinien
    ax1.grid(which='major', axis='x', linestyle='-', zorder=0, alpha=0.5)
    ax1.grid(which='minor', axis='x', linestyle='--', zorder=0, alpha=0.5)
    ax1.set_axisbelow(True)
    
    ## linien für median und mittel
    mediht=frame.xs('g', level='bz')['avht'].median()
    meanht=frame.xs('g', level='bz')['avht'].mean()
    medi_line=ax1.axvline(x=mediht,color='#FF7B00',linewidth=1,label=str(dec2min(mediht))+' mittel gesamt',zorder=0)
    mean_line=ax1.axvline(x=meanht,color='#FF0000',linewidth=1,label=str(dec2min(meanht))+' median gesamt',zorder=0)
    
    ## barchart für die Durchschnittszeiten
    g_aht=frame.xs('g', level='bz')['avht'].values
    g_att=frame.xs('g', level='bz')['avtt'].values
    g_aacw=frame.xs('g', level='bz')['avacw'].values

    bar_aht=ax1.barh(agents_ticks,g_aht,color=gcol,height=0.8,label='gesamt_aht')
    bar_att=ax1.barh(agents_ticks,g_att,color=verbcol,height=0.4,label='gesamt_att')
    bar_aacw=ax1.barh(agents_ticks,g_aacw,color=lostcol,left=g_att,height=0.4,label='gesamt_aacw')
       
    #####################################################
    ### subplot für die zahl der calls
    summe_calls_gesamt=frame.xs('g', level='bz')['an'].sum()
    
    ## Y-Achse wie im ersten subplot, aber alles auf der rechten Seite und ohne labels
    ax2.set_yticks(ax1.get_yticks())
    ax2.set_ylim(ax1.get_ylim())
    ax2.yaxis.tick_right()
    ax2.tick_params(axis='y',which='both',labelright=False)
    
    ## locators und formatters für die X-Achse
    ax2.xaxis.set_major_locator(AutoLocator())
    ax2.xaxis.set_minor_locator(AutoMinorLocator())
    ax2.xaxis.set_minor_formatter(FormatStrFormatter("%d"))
    for label in ax2.get_xticklabels(which='both'):          # lösung mit loop scheint am einfachsten zu sein...
        label.set_rotation(90) 
    for label in ax2.get_xticklabels(which='major'):
        label.set_fontsize(7)
        label.set_fontweight('bold')
    for label in ax2.get_xticklabels(which='minor'):
        label.set_fontsize(6)

    
    ## barchart für angenommene calls
    g_calls = frame.xs('g', level='bz')['an'].values
    k_calls = frame.xs('k', level='bz')['an'].values
    n_calls = frame.xs('n', level='bz')['an'].values
    bar_g=ax2.barh(agents_ticks,g_calls,color=gcol,height=0.4,label='anzahl calls gesamt')
    bar_k=ax2.barh(agents_ticks,k_calls,color=verbcol,height=0.15,label='anzahl calls kern')
    bar_n=ax2.barh(agents_ticks,n_calls,color=lostcol,left=k_calls,height=0.15,label='anzahl calls neben')
    
    ## X-Achse soll hier von rechts nach links gehen
    x2=ax2.get_xlim()    # normale x-achse
    x2_rev=(x2[1],x2[0]) # koordinaten umgedreht
    ax2.set_xlim(x2_rev) # und als neue achse gesetzt
    
    ## Hilfslinien
    ax2.grid(which='major', axis='x', linestyle='-', zorder=0, alpha=1)
    ax2.grid(which='minor', axis='x', linestyle='--', zorder=0, alpha=0.5)
    ax2.set_axisbelow(True)
    
    ## Summe der Calls in ein Eck
    ax2.text(0.5, 0.005, 'Summe: '+str(summe_calls_gesamt), transform = ax2.transAxes, color="black", fontsize='10',horizontalalignment='center')
    #fig.tight_layout()
    
    
    #####################################################
    ## Einstellungen, die die ganze fig betreffen
    
    ## Abstand der Plots zueinander
    plt.subplots_adjust(wspace=0.4)
    
    ## Titel für die beiden Subplots jeweils nach oben
    ax1.set_xlabel('Minuten')
    ax1.xaxis.set_label_coords(0.5,1.05)
    ax2.set_xlabel('Calls')
    ax2.xaxis.set_label_coords(0.5,1.05)
    
    titel='Aufschlüsselung nach Agent für den Zeitraum '+vonbis[0].strftime('%D')+' bis '+von_bis[1].strftime('%D')
    fig.suptitle(titel,fontsize=9)
    
    ## Legende
    handles, labels = ax1.get_legend_handles_labels()
    labels[0],labels[1],labels[2],labels[3],labels[4]=('mittel (ht)','median (ht)','Calls gesamt / HT','Calls Kernzeit / TT', 'Calls Nebenzeit / ACW')
    order=[2,3,4,0,1]; labels=[labels[x] for x in order]; handles=[handles[x] for x in order]
    
    fig.legend(handles,labels,loc='lower right', fontsize=7, bbox_to_anchor=(0.9,0.15),ncol=1)
    fig.set_facecolor('#000000')

In [134]:
agentplot(zeitraum,von_bis)